In [55]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import numpy
import scipy
import matplotlib
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import pytz
import multiprocessing as mpp
import pickle
import os
import sys
import json
import subprocess
#
# TODO: phase out unreferenced hpc_lib calls...
import hpc_lib
#
# system_name='mazama'
system_name='Sherlock_Owners'
groups_fname=None

#groups_fname='mazama_groups.json'
#
#data_file_name='data/sacct_mazama_out_2.out'
#data_file_name = 'data/mazama_usage_20200506_tool8.out'
#data_file_name = 'data/sacct_mazama_out_20200506a.out'
#data_file_name = 'data/sacct_serc_20200622.out'
#data_file_name = 'data/sacct_serc_20200622.out'
data_file_name = 'data/sacct_owners_out_3500489.out'
#
# as small(ish) test data set:
#data_file_name = 'data/sacct_mazama_20200701_20200718.out'

#data_file_name = 'data/serc_sacct_2020_06_29.out'

output_path = 'output/{}_HPC_analytics'.format(system_name)
#output_path = 'output/HPC_analytics'

In [ ]:
load_pickle = False
dump_pickle = True
max_rows = None
#max_rows = 1000
n_cpu=4
pkl_name = "{}.pkl".format(os.path.splitext(data_file_name)[0])
#
if load_pickle:
    with open(pkl_name, 'rb') as fin:
        sacct_obj=pickle.load(fin)
    #
else:
    sacct_obj = hpc_lib.SACCT_data_handler(data_file_name=data_file_name,
                    max_rows=max_rows, n_cpu=n_cpu, verbose=1)
    #
    if dump_pickle:
        with open(pkl_name, 'wb') as fout:
                #out_pkl = pickle.dump(sacct_demo.jobs_summary, fout)
                out_pkl = pickle.dump(sacct_obj, fout)
#

*** headers_rw:  User|Group|GID|JobName|JobID|JobIDRaw|Partition|State|Timelimit|NCPUS|NNodes|Submit|Eligible|Start|End|Elapsed|SystemCPU|UserCPU|TotalCPU|NTasks|CPUTimeRAW|Suspended|ReqGRES|AllocGRES|ReqTRES|AllocTRES|

*** load data:: headers:  ['User', 'Group', 'GID', 'JobName', 'JobID', 'JobIDRaw', 'Partition', 'State', 'Timelimit', 'NCPUS', 'NNodes', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed', 'SystemCPU', 'UserCPU', 'TotalCPU', 'NTasks', 'CPUTimeRAW', 'Suspended', 'ReqGRES', 'AllocGRES', 'ReqTRES', 'AllocTRES', 'JobID_parent']


In [ ]:
print("States: ", [s for s in set(sacct_obj.jobs_summary['State']) if not "CANCELLED" in s] )
#
#dT = .5
ix_pr  = sacct_obj.jobs_summary['State']=='PREEMPTED'
ix_npr = sacct_obj.jobs_summary['State']!='PREEMPTED'
ix_cpl = sacct_obj.jobs_summary['State']=='COMPLETED'
#
#
N_pr  = len(sacct_obj.jobs_summary[ix_pr])
N_npr = len(sacct_obj.jobs_summary[ix_npr])
print('*** preempted: {} vs {}, a/b={}'.format(N_pr, N_npr, N_pr/N_npr))


In [ ]:
print('** ', sacct_obj.data.dtype.names)
#print('** ', [s for s in set(sacct_objmazama.data['State']) if 'PR' in s])
#
preempted = (sacct_obj.jobs_summary[['JobID', 'NCPUS', 'NNodes', 'Submit', 'Start',
                              'End', 'Elapsed']][ix_pr])
preempted = preempted[numpy.argsort(preempted['JobID'])]
#
print('*** len: ', len(preempted))

#jsp = sacct_obj.

#print('** ', preempted[numpy.argsort(preempted['JobID'])][0:10])
#print('** ', preempted[0:10])
#pr_working = preempted[['JobID', 'NCPUS', 'NNodes', 'Submit', 'Start',
#                              'End', 'Elapsed']]
#
compl = sacct_obj.jobs_summary[['JobID', 'NCPUS', 'NNodes', 'Submit', 'Start',
                              'End', 'Elapsed']][ix_npr]

#compl = sacct_obj.jobs_summary[['JobID', 'NCPUS', 'NNodes', 'Submit', 'Start',
#                              'End', 'Elapsed']][sacct_obj.jobs_summary['State']!='PREEMPTED']

print(preempted[0:10])
print('** ', preempted.dtype.names)
#
print('** len(completed): ', len(compl))



### Preemption probabilities
- As a function of time
- Elapsed runtime, preceding preemption
- Count number $N(<t)$ for jobs that are (not) preempted, $P=N_p(t)/N_c(t)$

In [ ]:
fg = plt.figure(figsize=(14,8))
ax1 = fg.add_subplot('221')
ax2 = fg.add_subplot('222')
ax3 = fg.add_subplot('212')
#
ax1.set_ylabel('Probability $P$', size=16)
ax3.set_ylabel('Probability $P$', size=16)
ax1.set_xlabel('Elapsed runtime $\Delta t$', size=16)
ax2.set_xlabel('Elapsed runtime $\Delta t$', size=16)
#
hhs=[]

for nm, dta,ax in [("Preempted", preempted, ax1), ("Completed", compl, ax2)]:
    hh = ax.hist(dta['Elapsed']*24., bins=200, cumulative=True,
                  density=True, zorder=5, histtype='step')
    hhs+=[hh]
    #
    y_90=.9*numpy.max(hh[0])
    k_90=numpy.searchsorted(hh[0], y_90)
    #x_90 = .5*(hh[1][k_90]+hh[1][k_90+1])
    x_90 = numpy.mean(hh[1][k_90:k_90+2])
    ax.fill_between([0., x_90], [0.,0.], y_90, color='m', alpha=.2, zorder=11)
    #
    # 1 hour:
    dt=1.
    k1h = numpy.searchsorted(hh[1], dt)
    y1h = numpy.mean(hh[0][k1h-2:k1h])
    ax.fill_between([0., 1.], 0., y1h, color='m', alpha=.25, zorder=10 )

    ax.grid()
    ax.set_title("{}".format(nm))
    #
    print('{}: t_90: {}'.format(nm, x_90))
    print('{}: P(1hr): {}'.format(nm, y1h/numpy.max(hh[0])))
    #
print('** N_completed: {}'.format(len(compl)))
print('** N_pr: {}'.format(len(preempted)))
#
print('** P(re): ', len(preempted)/(len(preempted)+len(compl)))
#
# TODO: total probability distribution of preemption. so basically we want to add the cumulative histograms
#. of preempted and completed.To do that, we have to interpolate the two sets on to the same x-axis.
# a quick estimator is to just multiply by P(preempt).
#ax3.plot()
#print('** ', type(hh[0]))
max_bin = numpy.max([numpy.max(preempted['Elapsed']), numpy.max(compl['Elapsed'])])*24
print('*** ', max_bin)
bins = numpy.arange(0., max_bin,1.)
hhh = ax3.hist([compl['Elapsed']*24., preempted['Elapsed']*24.], bins=bins,
              histtype='barstacked', density=True, log=False, cumulative=True,
               bottom=0.0*numpy.ones(-1+len(bins)))
ax3.grid()
ax3.set_title('Preempted vs Not-preempted')
#
print('** ** ', numpy.shape(hhh[0]))
# note: if we're using the "barstacked" histogram, the second series is S2 = s1+s2.
#
P_pr=1.- (hhh[0][0]/hhh[0][1])
#
fg2=plt.figure(figsize=(12,4))
ax4 = fg2.add_subplot('111')
ax4.set_title('Preemption probability', size=16)
ax4.set_ylabel('Probability $P$', size=16)
ax4.set_xlabel('Elapsed runtime $\Delta t$', size=16)

#ax4a = ax4.twinx()
#
ax4.plot(hhh[1][1:], P_pr, ls='-', marker='.', label='$NP_k/N_k$')
# ax4.plot(bins[1:], hhh[0][0])
# ax4a.plot(bins[1:], hhh[0][1])
ax4.grid()
#
# let's just build this cum-prob manually:
dt_Np = numpy.zeros( (len(sacct_obj.jobs_summary),2))
#for t,s in sacct_obj.jobs_summary[numpy.argsort(sacct_mazama.jobs_summary['Elapsed'])]:
#    it s=='PREEMPTED'

ix_s = numpy.argsort(sacct_obj.jobs_summary['Elapsed'])
dt_Np[:,0] = sacct_obj.jobs_summary['Elapsed'][ix_s]
dt_Np[:,1] = numpy.cumsum((sacct_obj.jobs_summary['State'][ix_s]=='PREEMPTED').astype(int))
#
# /float(len(dt_Np))
ax4.plot(dt_Np[:,0]*24., dt_Np[:,1]/float(len(dt_Np)), ls='--', lw=2., label='$P(<\Delta_t)$')
#
# the hist and just counting cumulative measures match, if they're not normalized. just using the top
# of the stacked histogram doesnot seem to work quite right... because they are slightly different measures
# of the probability of PreEmption. gotta think about that a bit...
#hh = ax4.hist(preempted['Elapsed']*24., bins=100, cumulative=True,
#                  density=False, zorder=5, histtype='step')
ax4.legend(loc=0)
#
print('*** ***')
print(hhh[0][0][0:20])
print(hhh[0][1][0:20])

### Preemption prorabilities (total CPU time):
- Same as above, but compute as a function of totalcput time, not just elapsed.

In [ ]:
print('*** ', sacct_obj.jobs_summary.dtype.names)

In [ ]:
fg = plt.figure(figsize=(14,8))
ax1 = fg.add_subplot('221')
ax2 = fg.add_subplot('222')
ax3 = fg.add_subplot('212')
#
ax1.set_ylabel('Probability $P$', size=16)
ax3.set_ylabel('Probability $P$', size=16)
ax1.set_xlabel('Elapsed runtime $\Delta t$', size=16)
ax2.set_xlabel('Elapsed runtime $\Delta t$', size=16)
#
hhs=[]

for nm, dta,ax in [("Preempted", preempted, ax1), ("Completed", compl, ax2)]:
    # NOTE: TotalCPU, UserCPU, and SystemCPU -- until recently, were recorded as strings.
    #. they should the esame type as Elapsed, so we've replaced the processor function
    #. to convert them to float. BUT, until then, use Elapsed*NCPUS.
    #
    hh = ax.hist(dta['Elapsed']*dta['NCPUS']*24., bins=200, cumulative=True,
                  density=True, zorder=5, histtype='step')
    hhs+=[hh]
    #
    y_90=.9*numpy.max(hh[0])
    k_90=numpy.searchsorted(hh[0], y_90)
    #x_90 = .5*(hh[1][k_90]+hh[1][k_90+1])
    x_90 = numpy.mean(hh[1][k_90:k_90+2])
    ax.fill_between([0., x_90], [0.,0.], y_90, color='m', alpha=.2, zorder=11)
    #
    # 1 hour:
    dt=1.
    k1h = numpy.searchsorted(hh[1], dt)
    y1h = numpy.mean(hh[0][k1h-2:k1h])
    ax.fill_between([0., 1.], 0., y1h, color='m', alpha=.25, zorder=10 )

    ax.grid()
    ax.set_title("{}".format(nm))
    #
    print('{}: t_90: {}'.format(nm, x_90))
    print('{}: P(1hr): {}'.format(nm, y1h/numpy.max(hh[0])))
    #
print('** N_completed: {}'.format(len(compl)))
print('** N_pr: {}'.format(len(preempted)))
#
print('** P(re): ', len(preempted)/(len(preempted)+len(compl)))
#
# TODO: total probability distribution of preemption. so basically we want to add the cumulative histograms
#. of preempted and completed.To do that, we have to interpolate the two sets on to the same x-axis.
# a quick estimator is to just multiply by P(preempt).
#ax3.plot()
#print('** ', type(hh[0]))
max_bin = numpy.max([numpy.max(preempted['Elapsed']), numpy.max(compl['Elapsed'])])*24
print('*** ', max_bin)
bins = numpy.arange(0., max_bin,1.)
hhh = ax3.hist([compl['Elapsed']*24., preempted['Elapsed']*24.], bins=bins,
              histtype='barstacked', density=True, log=False, cumulative=True,
               bottom=0.0*numpy.ones(-1+len(bins)))
ax3.grid()
ax3.set_title('Preempted vs Not-preempted')
#
print('** ** ', numpy.shape(hhh[0]))
# note: if we're using the "barstacked" histogram, the second series is S2 = s1+s2.
#
P_pr=1.- (hhh[0][0]/hhh[0][1])
#
fg2=plt.figure(figsize=(12,4))
ax4 = fg2.add_subplot('111')
ax4.set_title('Preemption probability', size=16)
ax4.set_ylabel('Probability $P$', size=16)
ax4.set_xlabel('Elapsed runtime $\Delta t$', size=16)

#ax4a = ax4.twinx()
#
ax4.plot(hhh[1][1:], P_pr, ls='-', marker='.', label='$NP_k/N_k$')
# ax4.plot(bins[1:], hhh[0][0])
# ax4a.plot(bins[1:], hhh[0][1])
ax4.grid()
#
# let's just build this cum-prob manually:
dt_Np = numpy.zeros( (len(sacct_obj.jobs_summary),2))
#for t,s in sacct_obj.jobs_summary[numpy.argsort(sacct_mazama.jobs_summary['Elapsed'])]:
#    it s=='PREEMPTED'

ix_s = numpy.argsort(sacct_obj.jobs_summary['Elapsed'])
dt_Np[:,0] = sacct_obj.jobs_summary['Elapsed'][ix_s]
dt_Np[:,1] = numpy.cumsum((sacct_obj.jobs_summary['State'][ix_s]=='PREEMPTED').astype(int))
#
# /float(len(dt_Np))
ax4.plot(dt_Np[:,0]*24., dt_Np[:,1]/float(len(dt_Np)), ls='--', lw=2., label='$P(<\Delta_t)$')
#
# the hist and just counting cumulative measures match, if they're not normalized. just using the top
# of the stacked histogram doesnot seem to work quite right... because they are slightly different measures
# of the probability of PreEmption. gotta think about that a bit...
#hh = ax4.hist(preempted['Elapsed']*24., bins=100, cumulative=True,
#                  density=False, zorder=5, histtype='step')
ax4.legend(loc=0)
#
print('*** ***')
print(hhh[0][0][0:20])
print(hhh[0][1][0:20])

In [ ]:
wait_stats = sacct_obj.get_wait_stats()

In [ ]:
fg = plt.figure(figsize=(12,8))
ax1 = plt.subplot('111')
ax1.grid()
ax1.set_xlabel('NCPUS', size=16)
ax1.set_ylabel('Time [hours]')
#
ix = numpy.argsort(wait_stats['ncpus'])
ax1.plot(wait_stats['ncpus'][ix], 24.*wait_stats['median'][ix], ls='-', marker='.')

In [ ]:
#
delta_ts = 24.*(sacct_obj.jobs_summary['Start'] - sacct_obj.jobs_summary['Submit'])
ix = delta_ts>=0
#
fg = plt.figure(figsize=(12,8))
ax1 = plt.subplot('111')
ax1.grid()
#
ax2 = fg.add_axes([.3, .2, .55, .5])
ax2.grid()
#
hh = ax1.hist(delta_ts[ix], bins=numpy.arange(0., max(delta_ts[ix]), 1), density=True,
              cumulative=True, histtype='step', lw=3.0)
#
q_90=.9
k_90 = numpy.searchsorted(hh[0], q_90)
#print('*** k90: ', k_90)
x_90 = (hh[1][k_90]*(q_90-hh[0][k_90-1]) + hh[1][k_90+1]*(hh[0][k_90]-q_90))/(hh[0][k_90]-hh[0][k_90-1])

ax1.plot([min([ix][0]), max(delta_ts[ix])], [1., 1.], ls='--', lw=3., alpha=.8)
#
#ax1.set_ylim(.8,1.02)
ax1.set_xlim(-5, 200)
ax1.set_xlabel('Wait time (hours)', size=16)
ax1.set_ylabel('$P(>t)$', size=16)
#
ax2.plot(hh[1][1:11], hh[0][0:10], marker='o', ls='-', lw=3., zorder=12)
ax2.fill_between(hh[1][0:3], numpy.ones(3)*hh[0][1], hh[0][0],
                 color='m', alpha=.15, zorder=11)
#
ax2.fill_between([0,x_90], numpy.ones(2)*q_90, hh[0][0],
                 color='m', alpha=.15, zorder=11)
#
ax2.set_ylabel('$P(<t)$', size=16)
ax2.set_xlabel('time $t$ [hours]', size=16)

In [ ]:
print('** 2 hours: ', hh[0][0:2])
print('** 90%: ', x_90)

In [ ]:
print('*** ', numpy.mean(sacct_obj.jobs_summary['Elapsed'])*24.)


In [ ]:
print('*** hh[0]: ', hh[0][0:10])

In [ ]:
sacct_obj.__dict__.keys()

In [ ]:
#sacct_obj.jobs_summary.dtype.names
fg = plt.figure(figsize=(10,8))
ax1=plt.subplot('111')
#
total_cpu = hpc_lib.elapsed_time_2_day(sacct_obj.jobs_summary['TotalCPU'])*24.
hh=ax1.hist(total_cpu, bins=100)